In [10]:
from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import sklearn.metrics as metrics
import numpy as np
import torchvision
from torchvision import datasets, transforms
from kuzu import NetLin, NetFull, NetConv

def train(args, model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()        
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

def test(args, model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    conf_matrix = np.zeros((10,10)) # initialize confusion matrix
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            # sum up batch loss
            test_loss += F.nll_loss(output, target, reduction='sum').item()
            # get the index of the max log-probability
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()
            conf_matrix = conf_matrix + metrics.confusion_matrix(
                          pred.cpu(), target.cpu(), labels=[0,1,2,3,4,5,6,7,8,9])
        np.set_printoptions(precision=4, suppress=True)
        print(type(conf_matrix))
        print(conf_matrix)

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))




In [162]:

'''
loader = torch.utils.data.DataLoader(
    dataset,
    batch_size=8,
    num_workers=1,
    shuffle=True
)
print(loader)
print('dito',dataset.targets)
for batch_idx, (data, target) in enumerate(loader):
    print(batch_idx)
    print(data.shape)
    print(target)
    break
'''

# Fetch and load the test data
#testset = datasets.KMNIST(root='./Pokemon Dataset', train=False, download=False, transform=transform)
#test_loader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=False)

"\nloader = torch.utils.data.DataLoader(\n    dataset,\n    batch_size=8,\n    num_workers=1,\n    shuffle=True\n)\nprint(loader)\nprint('dito',dataset.targets)\nfor batch_idx, (data, target) in enumerate(loader):\n    print(batch_idx)\n    print(data.shape)\n    print(target)\n    break\n"

In [170]:
import codecs
csvReader = csv.reader(codecs.open('pokemon.csv', 'rU', 'utf-16'), delimiter='\t')
header = next(csvReader)
print(header)
print('---------')
types = dict()
english_name = 2
primary_type = 4
secondary_type = 5
for row in csvReader:
    name=row[english_name].lower().replace('\'','')
    types[name] = [row[primary_type],row[secondary_type]]
    #types[name] = [row[primary_type],'']
print(types)

['national_number', 'gen', 'english_name', 'japanese_name', 'primary_type', 'secondary_type', 'classification', 'percent_male', 'percent_female', 'height_m', 'weight_kg', 'capture_rate', 'base_egg_steps', 'hp', 'attack', 'defense', 'sp_attack', 'sp_defense', 'speed', 'abilities_0', 'abilities_1', 'abilities_2', 'abilities_hidden', 'against_normal', 'against_fire', 'against_water', 'against_electric', 'against_grass', 'against_ice', 'against_fighting', 'against_poison', 'against_ground', 'against_flying', 'against_psychict', 'against_bug', 'against_rock', 'against_ghost', 'against_dragon', 'against_dark', 'against_steel', 'against_fairy', 'is_sublegendary', 'is_legendary', 'is_mythical', 'evochain_0', 'evochain_1', 'evochain_2', 'evochain_3', 'evochain_4', 'evochain_5', 'evochain_6', 'gigantamax', 'mega_evolution', 'mega_evolution_alt', 'description']
---------
{'bulbasaur': ['grass', ''], 'ivysaur': ['grass', ''], 'venusaur': ['grass', ''], 'charmander': ['fire', ''], 'charmeleon': ['f

In [171]:
print(dataset.targets)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 

In [176]:
data_path = './Pokemon_Dataset'

dataset = torchvision.datasets.ImageFolder(
    root=data_path,
    transform=torchvision.transforms.ToTensor()
)

In [177]:
import os
import shutil

#match image name with the name in the csv file
nametype = dict()
#targets = []
sorted_path = "pokemon_type_sorted"
if not os.path.exists(sorted_path):
        os.makedirs(sorted_path)
for p in dataset.imgs:
    name = p[0].split('\\')[-1][:-4]
    print(name)
    if(len(name.split('-'))>1 and len(name.split('-')[1])>2):
        name = name.split('-')[0]
    print(p[0], types[name])
    #targets.append(types[name])
    for t in types[name]:
        if t == '':
            continue
        subsorted = sorted_path+"\\"+t
        if not os.path.exists(subsorted):
            os.makedirs(subsorted)
        shutil.copy2(p[0], subsorted+"\\"+name+".png") # complete target filename given
        nametype[p[0]] = types[name]

#dataset.targets = targets

abomasnow
./Pokemon_Dataset\1\abomasnow.png ['grass', '']
abra
./Pokemon_Dataset\1\abra.png ['psychic', '']
absol
./Pokemon_Dataset\1\absol.png ['dark', '']
accelgor
./Pokemon_Dataset\1\accelgor.png ['bug', '']
aegislash-shield
./Pokemon_Dataset\1\aegislash-shield.png ['steel', '']
aerodactyl
./Pokemon_Dataset\1\aerodactyl.png ['rock', '']
aggron
./Pokemon_Dataset\1\aggron.png ['steel', '']
aipom
./Pokemon_Dataset\1\aipom.png ['normal', '']
alakazam
./Pokemon_Dataset\1\alakazam.png ['psychic', '']
alcremie
./Pokemon_Dataset\1\alcremie.png ['fairy', '']
alomomola
./Pokemon_Dataset\1\alomomola.png ['water', '']
altaria
./Pokemon_Dataset\1\altaria.png ['dragon', '']
amaura
./Pokemon_Dataset\1\amaura.png ['rock', '']
ambipom
./Pokemon_Dataset\1\ambipom.png ['normal', '']
amoonguss
./Pokemon_Dataset\1\amoonguss.png ['grass', '']
ampharos
./Pokemon_Dataset\1\ampharos.png ['electric', '']
anorith
./Pokemon_Dataset\1\anorith.png ['rock', '']
appletun
./Pokemon_Dataset\1\appletun.png ['grass', 

./Pokemon_Dataset\1\froslass.png ['ice', '']
frosmoth
./Pokemon_Dataset\1\frosmoth.png ['ice', '']
furfrou-natural
./Pokemon_Dataset\1\furfrou-natural.png ['normal', '']
furret
./Pokemon_Dataset\1\furret.png ['normal', '']
gabite
./Pokemon_Dataset\1\gabite.png ['dragon', '']
gallade
./Pokemon_Dataset\1\gallade.png ['psychic', '']
galvantula
./Pokemon_Dataset\1\galvantula.png ['bug', '']
garbodor
./Pokemon_Dataset\1\garbodor.png ['poison', '']
garchomp
./Pokemon_Dataset\1\garchomp.png ['dragon', '']
gardevoir
./Pokemon_Dataset\1\gardevoir.png ['psychic', '']
gastly
./Pokemon_Dataset\1\gastly.png ['ghost', '']
gastrodon-west
./Pokemon_Dataset\1\gastrodon-west.png ['water', '']
genesect
./Pokemon_Dataset\1\genesect.png ['bug', '']
gengar
./Pokemon_Dataset\1\gengar.png ['ghost', '']
geodude
./Pokemon_Dataset\1\geodude.png ['rock', '']
gible
./Pokemon_Dataset\1\gible.png ['dragon', '']
gigalith
./Pokemon_Dataset\1\gigalith.png ['rock', '']
girafarig
./Pokemon_Dataset\1\girafarig.png ['norma

./Pokemon_Dataset\1\orbeetle.png ['bug', '']
oricorio-baile
./Pokemon_Dataset\1\oricorio-baile.png ['fire', '']
oshawott
./Pokemon_Dataset\1\oshawott.png ['water', '']
pachirisu
./Pokemon_Dataset\1\pachirisu.png ['electric', '']
palkia
./Pokemon_Dataset\1\palkia.png ['water', '']
palossand
./Pokemon_Dataset\1\palossand.png ['ghost', '']
palpitoad
./Pokemon_Dataset\1\palpitoad.png ['water', '']
pancham
./Pokemon_Dataset\1\pancham.png ['fighting', '']
pangoro
./Pokemon_Dataset\1\pangoro.png ['fighting', '']
panpour
./Pokemon_Dataset\1\panpour.png ['water', '']
pansage
./Pokemon_Dataset\1\pansage.png ['grass', '']
pansear
./Pokemon_Dataset\1\pansear.png ['fire', '']
paras
./Pokemon_Dataset\1\paras.png ['bug', '']
parasect
./Pokemon_Dataset\1\parasect.png ['bug', '']
passimian
./Pokemon_Dataset\1\passimian.png ['fighting', '']
patrat
./Pokemon_Dataset\1\patrat.png ['normal', '']
pawniard
./Pokemon_Dataset\1\pawniard.png ['dark', '']
pelipper
./Pokemon_Dataset\1\pelipper.png ['water', '']
p

tyrogue
./Pokemon_Dataset\1\tyrogue.png ['fighting', '']
tyrunt
./Pokemon_Dataset\1\tyrunt.png ['rock', '']
umbreon
./Pokemon_Dataset\1\umbreon.png ['dark', '']
unfezant
./Pokemon_Dataset\1\unfezant.png ['normal', '']
unown
./Pokemon_Dataset\1\unown.png ['psychic', '']
ursaring
./Pokemon_Dataset\1\ursaring.png ['normal', '']
urshifu-single-strike
./Pokemon_Dataset\1\urshifu-single-strike.png ['fighting', '']
uxie
./Pokemon_Dataset\1\uxie.png ['psychic', '']
vanillish
./Pokemon_Dataset\1\vanillish.png ['ice', '']
vanillite
./Pokemon_Dataset\1\vanillite.png ['ice', '']
vanilluxe
./Pokemon_Dataset\1\vanilluxe.png ['ice', '']
vaporeon
./Pokemon_Dataset\1\vaporeon.png ['water', '']
venipede
./Pokemon_Dataset\1\venipede.png ['bug', '']
venomoth
./Pokemon_Dataset\1\venomoth.png ['bug', '']
venonat
./Pokemon_Dataset\1\venonat.png ['bug', '']
venusaur
./Pokemon_Dataset\1\venusaur.png ['grass', '']
vespiquen
./Pokemon_Dataset\1\vespiquen.png ['bug', '']
vibrava
./Pokemon_Dataset\1\vibrava.png ['

In [187]:
#print(dataset.targets)
data_path = sorted_path

dataset = torchvision.datasets.ImageFolder(
    root=data_path,
    transform=torchvision.transforms.ToTensor()
)
print(dataset)
print(dataset.class_to_idx)

Dataset ImageFolder
    Number of datapoints: 898
    Root location: pokemon_type_sorted
    StandardTransform
Transform: ToTensor()
{'bug': 0, 'dark': 1, 'dragon': 2, 'electric': 3, 'fairy': 4, 'fighting': 5, 'fire': 6, 'flying': 7, 'ghost': 8, 'grass': 9, 'ground': 10, 'ice': 11, 'normal': 12, 'poison': 13, 'psychic': 14, 'rock': 15, 'steel': 16, 'water': 17}


In [188]:
#https://www.machinecurve.com/index.php/2021/02/03/how-to-use-k-fold-cross-validation-with-pytorch/

from sklearn.model_selection import KFold

# Configuration options
k_folds = 5
loss_function = nn.CrossEntropyLoss()

# For fold results
results = {}

# Set fixed random number seed
#torch.manual_seed(42)

# Define the K-fold Cross Validator
kfold = KFold(n_splits=k_folds, shuffle=True)


# Start print
print('--------------------------------')

# K-fold Cross Validation model evaluation
for fold, (train_ids, test_ids) in enumerate(kfold.split(dataset)):

    # Print
    print(f'FOLD {fold}')
    print('--------------------------------')
    
    #print(train_ids, test_ids)

    # Sample elements randomly from a given list of ids, no replacement.
    train_subsampler = torch.utils.data.SubsetRandomSampler(train_ids)
    test_subsampler = torch.utils.data.SubsetRandomSampler(test_ids)
    #print(train_subsampler, test_subsampler)
    # Define data loaders for training and testing data in this fold
    trainloader = torch.utils.data.DataLoader(
                      dataset, 
                      batch_size=10, sampler=train_subsampler)
    testloader = torch.utils.data.DataLoader(
                      dataset,
                      batch_size=10, sampler=test_subsampler)
    

--------------------------------
FOLD 0
--------------------------------
FOLD 1
--------------------------------
FOLD 2
--------------------------------
FOLD 3
--------------------------------
FOLD 4
--------------------------------


In [189]:
def reset_weights(m):
  '''
    Try resetting model weights to avoid
    weight leakage.
  '''
  for layer in m.children():
    if hasattr(layer, 'reset_parameters'):
        print(f'Reset trainable parameters of layer = {layer}')
        layer.reset_parameters()

class SimpleConvNet(nn.Module):
    '''
    Simple Convolutional Neural Network
    '''
    def __init__(self):
        super(SimpleConvNet, self).__init__()
        # INSERT CODE HERE
        self.conv1 = nn.Conv2d(3, 6, 5,  stride=3)
        self.conv2 = nn.Conv2d(6, 12, 5,  stride=3)
        self.fc1 = nn.Linear(31212, 1000)
        self.fc2 = nn.Linear(1000, 50)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        #print(x.shape)

        x = x.view(x.shape[0], 31212)

        #print(x.shape)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        #print(x.shape)
        probs = F.log_softmax(x)
        return probs # CHANGE CODE HERE
  
# https://www.machinecurve.com/index.php/2021/02/03/how-to-use-k-fold-cross-validation-with-pytorch/
num_epochs = 5
# Init the neural network
network = SimpleConvNet()
network.apply(reset_weights)

# Initialize optimizer
optimizer = torch.optim.Adam(network.parameters(), lr=1e-4)

# Run the training loop for defined number of epochs
for epoch in range(0, num_epochs):

  # Print epoch
  print(f'Starting epoch {epoch+1}')

  # Set current loss value
  current_loss = 0.0

  # Iterate over the DataLoader for training data
  for i, data in enumerate(trainloader, 0):

    # Get inputs
    inputs, targets = data

    #print(inputs, "targets:",targets)
    # Zero the gradients
    optimizer.zero_grad()

    # Perform forward pass
    outputs = network(inputs)

    # Compute loss
    loss = loss_function(outputs, targets)

    # Perform backward pass
    loss.backward()

    # Perform optimization
    optimizer.step()

    # Print statistics
    current_loss += loss.item()
    if i % 500 == 499:
        print('Loss after mini-batch %5d: %.3f' %
              (i + 1, current_loss / 500))
        current_loss = 0.0

# Process is complete.
print('Training process has finished. Saving trained model.')

# Print about testing
print('Starting testing')

# Saving the model
save_path = f'./model-fold-{fold}.pth'
torch.save(network.state_dict(), save_path)

# Evaluationfor this fold
correct, total = 0, 0
with torch.no_grad():

  # Iterate over the test data and generate predictions
  for i, data in enumerate(testloader, 0):

    # Get inputs
    inputs, targets = data

    # Generate outputs
    outputs = network(inputs)

    # Set total and correct
    _, predicted = torch.max(outputs.data, 1)
    total += targets.size(0)
    print(predicted, " :: ", targets)
    correct += (predicted == targets).sum().item()

  # Print accuracy
  print('Accuracy for fold %d: %d %%' % (fold, 100.0 * correct / total))
  print('--------------------------------')
  results[fold] = 100.0 * (correct / total)

# Print fold results
print(f'K-FOLD CROSS VALIDATION RESULTS FOR {k_folds} FOLDS')
print('--------------------------------')
sum = 0.0
for key, value in results.items():
    print(f'Fold {key}: {value} %')
    sum += value
print(f'Average: {sum/len(results.items())} %')

Reset trainable parameters of layer = Conv2d(3, 6, kernel_size=(5, 5), stride=(3, 3))
Reset trainable parameters of layer = Conv2d(6, 12, kernel_size=(5, 5), stride=(3, 3))
Reset trainable parameters of layer = Linear(in_features=31212, out_features=1000, bias=True)
Reset trainable parameters of layer = Linear(in_features=1000, out_features=50, bias=True)
Starting epoch 1


<ipython-input-189-ad936121151d>:34: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  probs = F.log_softmax(x)


Starting epoch 2
Starting epoch 3
Starting epoch 4
Starting epoch 5
Training process has finished. Saving trained model.
Starting testing
tensor([17,  4,  9, 17, 12,  4,  5,  9,  5, 13])  ::  tensor([11, 15,  9,  4,  0, 11,  5,  9, 12, 17])
tensor([17,  6,  0, 14, 14, 17, 12, 14,  0,  0])  ::  tensor([14,  6,  0, 17,  6, 11, 17, 14,  9, 12])
tensor([12, 13, 17,  5, 15, 12, 17, 12, 14, 17])  ::  tensor([ 0, 17, 15,  8, 14, 14, 17, 10, 17,  9])
tensor([17, 13, 17, 14, 17, 17, 12, 14,  6, 12])  ::  tensor([ 3, 17, 16,  2, 12, 16, 17, 16,  6,  8])
tensor([17, 12,  0, 17, 12, 17, 17, 17, 13, 17])  ::  tensor([12,  6, 10, 15,  9,  9, 17, 15,  1, 13])
tensor([17,  0, 12, 17, 17, 17, 16, 12, 17, 17])  ::  tensor([ 5, 17,  4,  2, 15,  3, 11,  3,  9, 17])
tensor([17,  6,  0, 12, 12, 12, 17, 14,  0, 14])  ::  tensor([ 4,  9,  9,  5,  9, 14, 17,  3,  0, 15])
tensor([17, 17, 17, 17, 14, 12,  6, 14, 12, 17])  ::  tensor([17, 17, 11, 12,  5,  6,  6,  0, 12, 11])
tensor([ 4, 17,  0,  9,  4, 13,  0, 17